In [1]:
from dotenv import load_dotenv
load_dotenv() # .env 파일에 정의된 환경변수들을 로드합니다.

True

In [2]:
import os  # 운영체제와 상호작용 (파일 경로 등)
from glob import glob # 특정 패턴에 맞는 파일 목록 가져오기

from pprint import pprint # 복잡한 데이터 구조를 보기 좋게 출력
import json # JSON 데이터 처리

## 1. 다양한 문서 형식 처리하기

- RAG 시스템의 성능은 입력 데이터의 품질에 크게 좌우됨. 
- LangChain은 다양한 형식의 문서를 일관된 `Document` 객체로 변환하는 'Document Loaders'를 제공
-  `Document` 객체는 주로 `page_content` (텍스트 내용)와 `metadata` (출처, 페이지 번호 등의 부가 정보)로 구성

### 1.1 PDF 문서 

- `PyPDFLoader`는 PDF 파일에서 텍스트를 추출
- 각 페이지는 별도의 `Document` 객체로 로드
- PDF의 텍스트 레이어가 잘 구성되어 있을수록 추출 품질이 좋음
- 이미지 기반 PDF의 경우 OCR 처리가 필요할 수 있으며, `PyPDFLoader`는 기본적으로 텍스트 기반 PDF에 적합

**장점:**
- 가장 널리 사용되는 문서 형식 중 하나
- 페이지 번호 등 메타데이터가 자동으로 추출되어 출처 표시에 유용

**단점:**
- 복잡한 레이아웃, 표, 이미지가 많은 PDF는 텍스트 추출이 완벽하지 않을 수 있음
- 스캔된 이미지로만 구성된 PDF는 텍스트 추출이 불가능 (별도 OCR 필요).
- `UnstructuredPDFLoader`나 `PyPDFium2Loader` 등 다른 로더를 사용하면 더 나은 결과를 얻을 수도 있음


In [3]:
from langchain_community.document_loaders import PyPDFLoader

# './data/transformer.pdf' 파일을 로드합니다.
pdf_loader = PyPDFLoader('./data/transformer.pdf')
pdf_docs = pdf_loader.load() # Document 객체의 리스트로 반환

print(f"PDF에서 로드된 Document 수: {len(pdf_docs)}")

PDF에서 로드된 Document 수: 15


### 1.2 웹 문서 
- `WebBaseLoader`는 지정된 URL의 웹 페이지 내용 
- HTML을 파싱하여 텍스트 콘텐츠를 추출


**장점:**
- 웹 상의 방대한 정보를 쉽게 가져올 수 있음
- 여러 URL을 한 번에 처리할 수 있음

**단점:**
- JavaScript로 동적으로 렌더링되는 콘텐츠는 제대로 가져오지 못할 수 있음 
(이 경우 `PlaywrightURLLoader`나 `SeleniumURLLoader` 등 브라우저 자동화 기반 로더 고려)
- 광고, 메뉴, 푸터 등 불필요한 내용이 함께 추출될 수 있어 후처리가 필요할 가능성 존재
- 웹사이트 구조에 따라 추출 품질이 달라질 수 있음

In [4]:
from langchain_community.document_loaders import WebBaseLoader

# 여러 웹 페이지 URL을 리스트로 전달하여 로드합니다.
web_loader = WebBaseLoader(["https://python.langchain.com/", "https://js.langchain.com/"])

web_docs = web_loader.load()

print(f"웹에서 로드된 Document 수: {len(web_docs)}")

USER_AGENT environment variable not set, consider setting it to identify your requests.


웹에서 로드된 Document 수: 2


In [5]:
# 첫 번째 웹 Document의 메타데이터 확인
# 'source'는 URL, 'title', 'description', 'language' 등 웹페이지 메타 정보가 포함될 수 있습니다.
web_docs[0].metadata

{'source': 'https://python.langchain.com/',
 'title': 'Introduction | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain',
 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).',
 'language': 'en'}

In [6]:
# 첫 번째 웹 Document의 페이지 내용 (일부만 출력)
print(web_docs[0].page_content[:500])






Introduction | ü¶úÔ∏èüîó LangChain






Skip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith.  Join our team!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1üí¨SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augment


### 1.3 JSON 파일 

- `JSONLoader`는 JSON 파일에서 데이터를 로드 
- `jq_schema` 파라미터를 사용하여 JSON 구조 내 특정 경로의 데이터를 추출
- `jq`는 JSON 데이터를 다루는 강력한 커맨드라인 도구이며, LangChain은 이 `jq`의 문법을 활용

**`jq_schema` 설명:**
- `.` : 현재 객체 또는 배열을 의미
- `.key` : 현재 객체에서 'key'에 해당하는 값을 선택
- `.array[]` : 'array'라는 키를 가진 배열의 모든 요소를 선택
- `.array[].field` : 'array' 배열의 각 요소(객체)에서 'field' 키의 값을 선택

**`text_content=True` vs `text_content=False`:**
- `text_content=True` (기본값): `jq_schema`로 선택된 값이 문자열이 아닐 경우 문자열로 변환하여 `page_content`에 저장, 선택된 값이 문자열이라면 그대로 사용
- `text_content=False`: `jq_schema`로 선택된 값을 JSON 문자열 형태로 `page_content`에 저장하며, 객체나 배열이 선택되면 해당 구조가 그대로 문자열화 되어 저장

**장점:**
- 정형화된 데이터를 유연하게 처리 가능
- `jq_schema`를 통해 필요한 정보만 정확히 추출 가능

**단점:**
- `jq_schema` 문법에 대한 이해가 필요
- 복잡한 중첩 구조의 JSON의 경우 `jq_schema`가 길어질 수 있음.

In [7]:
from langchain_community.document_loaders import JSONLoader

json_loader = JSONLoader(
    file_path="./data/kakao_chat.json",
    jq_schema=".messages[].content",    # messages 배열의 각 요소(객체)에서 'content' 필드 값을 추출
    text_content=True,                  # 추출된 content 필드가 텍스트이므로 True (사실상 문자열이면 True/False 큰 차이 없음)
)

json_docs = json_loader.load()

print("문서의 수:", len(json_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", json_docs[0].metadata) # 기본적인 파일 정보만 포함
print("-" * 50)
print("처음 문서의 내용: \n", json_docs[0].page_content) # 'content' 필드의 텍스트만 추출됨

문서의 수: 5
--------------------------------------------------
처음 문서의 메타데이터: 
 {'source': 'D:\\git\\langchain\\data\\kakao_chat.json', 'seq_num': 1}
--------------------------------------------------
처음 문서의 내용: 
 안녕하세요 여러분, 오늘 회의 시간 확인차 연락드립니다.


In [8]:
from langchain_community.document_loaders import JSONLoader

json_loader = JSONLoader(
    file_path="./data/kakao_chat.json",
    jq_schema=".messages[]",    # messages 배열의 각 객체 전체를 추출
    text_content=False,          # 객체를 JSON 문자열 형태로 page_content에 저장
)

json_docs_obj_as_str = json_loader.load()

print("문서의 수:", len(json_docs_obj_as_str))
print("-" * 50)
print("처음 문서의 메타데이터: \n", json_docs_obj_as_str[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", json_docs_obj_as_str[0].page_content) # 객체가 문자열화 되어 저장, 유니코드 이스케이프 주의

문서의 수: 5
--------------------------------------------------
처음 문서의 메타데이터: 
 {'source': 'D:\\git\\langchain\\data\\kakao_chat.json', 'seq_num': 1}
--------------------------------------------------
처음 문서의 내용: 
 {"sender": "\uae40\ucca0\uc218", "timestamp": "2023-09-15 09:30:22", "content": "\uc548\ub155\ud558\uc138\uc694 \uc5ec\ub7ec\ubd84, \uc624\ub298 \ud68c\uc758 \uc2dc\uac04 \ud655\uc778\ucc28 \uc5f0\ub77d\ub4dc\ub9bd\ub2c8\ub2e4."}


**유니코드 디코딩 및 `Document` 객체 재구성**
- `text_content=False`로 객체를 문자열화했을 때 한글이 `\uXXXX` 형태로 나올 수 있음 
- 이를 다시 파싱하여 사람이 읽을 수 있는 형태로 만들고, 필요하다면 `Document` 객체로 재구성할 수 있음
- `json.loads()`로 문자열을 파이썬 객체로 변환하고, `json.dumps(..., ensure_ascii=False)`를 사용해 한글이 깨지지 않는 JSON 문자열 만듬

In [9]:
# 유니코드 디코딩 (한글 문자들이 유니코드 이스케이프 시퀀스로 인코딩되어 있을 경우)
from langchain_core.documents import Document

decoded_json_docs = []
for doc in json_docs_obj_as_str: # 이전 셀에서 생성한 json_docs_obj_as_str 사용
    # page_content (JSON 문자열)를 파이썬 딕셔너리로 변환
    decoded_data = json.loads(doc.page_content)

    # 파이썬 딕셔너리를 한글이 유지되는 JSON 문자열로 변환하여 새로운 Document 객체 생성
    # 또는 필요한 내용만 추출하여 page_content 구성 가능
    document_obj = Document(
        page_content=json.dumps(decoded_data, ensure_ascii=False),
        metadata=doc.metadata
    )
    decoded_json_docs.append(document_obj)

print("문서의 수:", len(decoded_json_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", decoded_json_docs[0].metadata)
print("-" * 50)
print("처음 문서의 내용 (디코딩 및 재구성 후): \n", decoded_json_docs[0].page_content)

문서의 수: 5
--------------------------------------------------
처음 문서의 메타데이터: 
 {'source': 'D:\\git\\langchain\\data\\kakao_chat.json', 'seq_num': 1}
--------------------------------------------------
처음 문서의 내용 (디코딩 및 재구성 후): 
 {"sender": "김철수", "timestamp": "2023-09-15 09:30:22", "content": "안녕하세요 여러분, 오늘 회의 시간 확인차 연락드립니다."}


**`metadata_func`를 이용한 커스텀 메타데이터 추출**
- `JSONLoader`의 `metadata_func` 파라미터를 사용하면 원본 JSON 레코드에서 원하는 정보를 추출하여 `Document` 객체의 `metadata`에 추가할 수 있음. 
- RAG 시 검색된 청크의 출처나 추가 정보를 제공하는 데 유용.

- `content_key`: `jq_schema`로 선택된 각 아이템(주로 객체)에서 실제 `page_content`로 사용할 텍스트가 담긴 필드의 키를 지정

**JSONL (JSON Lines) 파일 처리**

JSONL은 각 줄이 독립적인 JSON 객체인 형식입니다. `JSONLoader`에서 `json_lines=True`로 설정하여 처리 가능

In [10]:
# JSONL 파일 로드하기 - content_key 사용
jsonl_loader_content_key = JSONLoader(
    file_path="./data/kakao_chat.jsonl",
    jq_schema=".",          # 각 줄(JSON 객체) 전체를 대상으로 함
    content_key="content",  # 객체에서 'content' 필드 값을 page_content로 사용
    json_lines=True,      # JSONL 형식임을 명시
)

jsonl_docs_content_key = jsonl_loader_content_key.load()

print("문서의 수:", len(jsonl_docs_content_key))
print("-" * 50)
print("처음 문서의 메타데이터 (커스텀 없음): \n", jsonl_docs_content_key[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", jsonl_docs_content_key[0].page_content)

문서의 수: 5
--------------------------------------------------
처음 문서의 메타데이터 (커스텀 없음): 
 {'source': 'D:\\git\\langchain\\data\\kakao_chat.jsonl', 'seq_num': 1}
--------------------------------------------------
처음 문서의 내용: 
 안녕하세요 여러분, 오늘 회의 시간 확인차 연락드립니다.


In [11]:
# 메타데이터 추가하기
def metadata_func(record: dict, metadata: dict) -> dict:
    # record: jq_schema에 의해 선택된 JSON 객체 (여기서는 messages 배열의 각 요소)
    # metadata: 기본적으로 생성되는 메타데이터 (source, seq_num 등)
    metadata["sender"] = record.get("sender")       # record에서 'sender' 값을 가져와 메타데이터에 추가
    metadata["timestamp"] = record.get("timestamp") # record에서 'timestamp' 값을 가져와 메타데이터에 추가
    return metadata


json_loader_with_meta = JSONLoader(
    file_path="./data/kakao_chat.json",
    jq_schema=".messages[]",  # messages 배열의 각 객체를 대상으로 함
    content_key="content",    # 각 객체에서 'content' 필드 값을 page_content로 사용
    metadata_func=metadata_func, # 위에서 정의한 함수를 메타데이터 추출에 사용
)

json_docs_with_meta = json_loader_with_meta.load()

print("문서의 수:", len(json_docs_with_meta))
print("-" * 50)
print("처음 문서의 메타데이터 (커스텀 추가됨): \n", json_docs_with_meta[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", json_docs_with_meta[0].page_content)

문서의 수: 5
--------------------------------------------------
처음 문서의 메타데이터 (커스텀 추가됨): 
 {'source': 'D:\\git\\langchain\\data\\kakao_chat.json', 'seq_num': 1, 'sender': '김철수', 'timestamp': '2023-09-15 09:30:22'}
--------------------------------------------------
처음 문서의 내용: 
 안녕하세요 여러분, 오늘 회의 시간 확인차 연락드립니다.


In [12]:
# 메타데이터 추가하기 (위에서 정의한 metadata_func 재사용)
jsonl_loader_with_meta = JSONLoader(
    file_path="./data/kakao_chat.jsonl",
    jq_schema=".",                 # 각 줄(JSON 객체) 전체를 대상으로 함
    content_key="content",         # 객체에서 'content' 필드 값을 page_content로 사용
    metadata_func=metadata_func,   # 위에서 정의한 함수를 메타데이터 추출에 사용
    json_lines=True,             # JSONL 형식임을 명시
)

jsonl_docs_with_meta = jsonl_loader_with_meta.load()

print("문서의 수:", len(jsonl_docs_with_meta))
print("-" * 50)
print("처음 문서의 메타데이터 (커스텀 추가됨): \n", jsonl_docs_with_meta[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", jsonl_docs_with_meta[0].page_content)

# json_docs 변수를 이후 정규표현식 분할 예제에서 사용하기 위해 jsonl_docs_with_meta로 할당
json_docs = jsonl_docs_with_meta 

문서의 수: 5
--------------------------------------------------
처음 문서의 메타데이터 (커스텀 추가됨): 
 {'source': 'D:\\git\\langchain\\data\\kakao_chat.jsonl', 'seq_num': 1, 'sender': '김철수', 'timestamp': '2023-09-15 09:30:22'}
--------------------------------------------------
처음 문서의 내용: 
 안녕하세요 여러분, 오늘 회의 시간 확인차 연락드립니다.


### 1.4  CSV 문서 

- `CSVLoader`는 CSV파일을 로드
- 각 행(row)을 하나의 `Document` 객체로 만듬
-  `page_content`는 모든 열의 내용을 "컬럼명: 값" 형태로 조합하여 생성됩니다.

**주요 파라미터:**
- `source_column`: 특정 열의 값을 `metadata`의 `source` 키 값으로 사용하도록 지정할 수 있음
- `csv_args`: `delimiter`, `quotechar` 등 CSV 파싱 관련 추가 인자를 딕셔너리 형태로 전달할 수 있음
- `encoding`: 파일 인코딩을 지정합니다 (예: 'utf-8', 'cp949').

**장점:**
- 표 형식의 데이터를 쉽게 로드할 수 있음
- 각 행이 개별적인 정보 단위를 나타낼 때 유용

**단점:**
- 긴 텍스트나 복잡한 구조의 데이터를 표현하기에는 적합하지 않을 수 있음
- 기본적으로 모든 열의 내용이 `page_content`에 포함되므로, 특정 열만 사용하고 싶다면 로드 후 추가 처리가 필요하거나, `CSVLoader`의 `content_columns` (또는 유사한 기능, LangChain 버전에 따라 다를 수 있음) 파라미터를 확인해함
(현재 `CSVLoader`에는 명시적인 `content_columns` 파라미터가 없을 수 있으므로, 로드 후 Document 리스트를 순회하며 `page_content`를 재구성하는 방법이 일반적일 수 있음)

In [13]:
from langchain_community.document_loaders.csv_loader import CSVLoader

csv_loader = CSVLoader("./data/kbo_teams_2023.csv", encoding='utf-8') # 한글 포함 시 encoding 명시
csv_docs = csv_loader.load()

print("문서의 수:", len(csv_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", csv_docs[0].metadata) # 'source'는 파일 경로, 'row'는 행 번호 (0부터 시작)
print("-" * 50)
print("처음 문서의 내용: \n", csv_docs[0].page_content) # 모든 컬럼이 "컬럼명: 값" 형태로 조합됨

문서의 수: 10
--------------------------------------------------
처음 문서의 메타데이터: 
 {'source': './data/kbo_teams_2023.csv', 'row': 0}
--------------------------------------------------
처음 문서의 내용: 
 Team: KIA 타이거즈
City: 광주
Founded: 1982
Home Stadium: 광주-기아 챔피언스 필드
Championships: 11
Introduction: KBO 리그의 전통 강호로, 역대 최다 우승 기록을 보유하고 있다. '타이거즈 스피릿'으로 유명하며, 양현종, 안치홍 등 스타 선수들을 배출했다. 광주를 연고로 하는 유일한 프로야구팀으로 지역 사랑이 강하다.
